In [2]:
import datetime
import io
import re
import numpy as np
import pandas as pd
import boto3
import pyarrow.parquet as pq
import s3fs
import math

#### Fetch data

In [3]:
# realized sales
def to_uri(bucket, key):
    """
    List all files under a S3 bucket
    :param bucket: (string) name of the S3 bucket
    :param key: (string) S3 key
    :return: (string) URI format
    """
    return f's3://{bucket}/{key}'

def read_multipart_parquet_s3(bucket, dir_path, prefix_filename ='part-'):
    """
    Read a multipart parquet file (splitted) hosted on a S3 bucket, load and return as pandas dataframe. Note that only
    files with name starting with <prefix_filename> value are taken into account.
    :param bucket: (string) S3 source bucket
    :param dir_path: (string) full path to the folder that contains all parts within this S3 bucket
    :param prefix_filename: (string) Optional. Default is 'part-' which is what Spark generates. Only files with name
    starting with this value will be loaded
    :return: (pandas DataFrame) data loaded
    """
    fs = s3fs.S3FileSystem()
    s3_uri = to_uri(bucket, dir_path)
    data = pq.ParquetDataset(s3_uri, filesystem =fs).read().to_pandas(date_as_object = False)

    return data

def week_id_to_date(week_id):
    """
    Turn a Decathlon week id to date
    :param week_id: (int or pd.Series) the week id or pandas column of week ids
    :return: (pd.Timestamp or pd.Series) the date or pandas column of dates
    """
    assert isinstance(week_id, (int, np.integer, pd.Series, list))
    if isinstance(week_id, (int, np.integer)):
        assert is_iso_format(week_id), f"`week_id` {week_id} doesn't follow conform YYYYWW format"
        return pd.to_datetime(str(week_id) + '-0', format = '%G%V-%w') - pd.Timedelta(1, unit = 'W')
    else:
        if isinstance(week_id, (list)):
            week_id = pd.Series(week_id)
        for w in week_id:
            assert is_iso_format(w), f"Week_id {w} doesn't follow conform YYYYWW format"
        return pd.to_datetime(week_id.astype(str) + '-0', format = '%G%V-%w') - pd.Timedelta(1, unit = 'W')

def is_iso_format(week_id: int) -> bool:
    """
    Checks if `week` has conform YYYYWW ISO 8601 format.
    :param week_id: (int or pd.Series) the week id or pandas column of week ids
    :return: (bool) wether `week_id` follows the expected format
    """
    assert isinstance(week_id, (int, np.int32, np.int64)), "week_id must be of type integer."
    pattern = "^20[0-9]{2}(0[1-9]|[1-4][0-9]|5[0-3])$"
    is_iso_format = re.match(pattern, str(week_id))

    return is_iso_format is not None

def date_to_week_id(date):
    """
    Turn a date to Decathlon week id
    :param date: (str, pd.Timestamp or pd.Series) the date or pandas column of dates
    :return: (int or pd.Series) the week id or pandas column of week ids
    """
    assert isinstance(date, (str, pd.Timestamp, pd.Series, datetime.date))
    if isinstance(date, (str, pd.Timestamp, datetime.date)):
        date = pd.Timestamp(date)
        if date.dayofweek == 6:  # If sunday, replace by next monday to get the correct iso week
            date = date + pd.Timedelta(1, unit = 'D')
        week_id = int(str(date.isocalendar()[0]) + str(date.isocalendar()[1]).zfill(2))
        return week_id
    else:
        df = pd.DataFrame({'date': pd.to_datetime(date)})
        df['dow'] = df['date'].dt.dayofweek
        df.loc[df['dow'] == 6, 'date'] = df.loc[df['dow'] == 6, 'date'] + pd.Timedelta(1, unit = 'D')
        df['week_id'] = df['date'].apply(lambda x: int(str(x.isocalendar()[0]) + str(x.isocalendar()[1]).zfill(2)))
        return df['week_id']
    
def get_week_id(str_,step):
    str_ = str(str_)
    year = int(str_[:4])
    week = int(str_[4:])
    step = step - 1
    if week + step <= 52:
        if week+step < 10:
            return int(str(year) + '0' +str(week + step))
        else:
            return int(str(year) + str(week + step))
    else:
        round_ = math.floor((week + step) / 52)
        year = year + round_
        week = week + step - 52 * round_
        if week < 10:
            return int(str(year) + '0' + str(week))
        else:
            return int(str(year) + str(week))

In [4]:
def apply_lockdowns_reconstruction(df_sales, df_sales_reconstructed):
    """
    Replaces the lockdown weeks of `df_sales` with the reconstructed ones of `df_sales_reconstructed`.

    Args:
        df_sales (pd.DataFrame): The initial sales dataframe
        df_sales_reconstructed (pd.DataFrame): The reconstructed sales dataframe

    Returns:
        pd.DataFrame with the same structure as `df_sales`, whose lockdown weeks have been
        replaced by the ones in `df_sales_reconstructed`
    """
    df_sales = pd.merge(df_sales, df_sales_reconstructed, how = 'left')
    df_sales['sales_quantity'] = np.where(df_sales['sales_quantity_reconstructed'].notnull(),
                                          df_sales['sales_quantity_reconstructed'],
                                          df_sales['sales_quantity'])
    df_sales.drop(columns='sales_quantity_reconstructed', inplace = True)
    return df_sales

def pad_to_cutoff(df_ts: pd.DataFrame,
                  cutoff: int,
                  col: str = 'sales_quantity',
                  ) -> pd.DataFrame:
    """
    Forward fills with zeros time series in Pandas DataFrame up to week cutoff.

    The function will complete the dataframe for all models (`model_id`) with a frequency
    of 1 week, up to week_id cutoff (excluding it) on the column `col` with value = 0.
    The input dataframe must include columns ['model_id', 'week_id', 'date'].

    Args:
        df_ts (pd.DataFrame): Timeseries DataFrame
        cutoff (int): ISO 8601 Format Week id (YYYYWW) to forward fill to
        col (str): Name of the column to fill

    Returns:
        df (pd.DataFrame): Padded DataFrame
    """
    assert is_iso_format(cutoff)
    assert isinstance(cutoff, (int, np.int64))
    assert isinstance(df_ts, pd.DataFrame)
    assert pd.api.types.is_datetime64_any_dtype(df_ts['date'])
    assert set(df_ts.columns) == set(['model_id', 'week_id', 'date', col])

    # Limiting dataset to avoid errors if we have data further than cutoff
    df_ts = df_ts[df_ts['week_id'] < cutoff]
    # Add the cutoff weekend to all models to put a limit for the bfill
    models = df_ts['model_id'].unique()
    test_cutoff_date = week_id_to_date(cutoff)
    md, cu = pd.core.reshape.util.cartesian_product([models, [cutoff]])
    df_ts_tail = pd.DataFrame({"model_id": md, "week_id": cu})
    df_ts_tail['date'] = test_cutoff_date
    df_ts_tail[col] = 0
    df = df_ts.append(df_ts_tail)

    # Backfill for the cutoff week
    df = df.set_index('date').groupby(['model_id']).resample('1W').asfreq().fillna(0)
    
    # Getting the df back to its original form
    df.drop(['model_id'], axis = 1, inplace = True)
    df.reset_index(inplace = True)
    df['week_id'] = date_to_week_id(df['date'])

    # Getting rid of the cutoff week
    df = df[df['week_id'] < cutoff]
    
    if col == 'sales_quantity':
        df[col] = df[col].astype(int)
    
    return df

In [8]:
cutoff = 202138
# shutdown reconstructed sales
reconstructed_sales_lockdowns = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                           'global/reconstructed_sales_lockdowns.parquet', prefix_filename ='part-')

# Sales refining
df_sales = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                     'global/model_week_sales', prefix_filename ='part-')
df_sales = df_sales.groupby(by=['model_id','week_id','date'],as_index=False).agg({'sales_quantity':'sum'})
df_sales = df_sales[df_sales['week_id'] < cutoff]
df_sales = apply_lockdowns_reconstruction(df_sales, reconstructed_sales_lockdowns)
df_weekly_sales = df_sales

# MRP refining
df_mrp = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                   'global/model_week_mrp', prefix_filename ='part-')
df_mrp = df_mrp[df_mrp['week_id'] == cutoff]

# Tree refining
df_tree = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_tree', prefix_filename ='part-')
df_tree = df_tree[df_tree['week_id'] == cutoff]

# Limiting Sales data to MRP active models
df_sales = pd.merge(df_sales, df_mrp.loc[df_mrp['is_mrp_active'], ['model_id']])

# Pad to cutoff Sales data
df_sales = pad_to_cutoff(df_sales, cutoff)

In [9]:
# for new model pool
df_sales.head(2)

,model_id,date,week_id,sales_quantity
0,720719,2016-01-03,201601,45
1,720719,2016-01-10,201602,55


In [10]:
# for old model pool
df_weekly_sales.head(2)

,model_id,week_id,date,sales_quantity
0,18259,201614,2016-04-03,1.0
1,94885,201716,2017-04-16,1.0


In [11]:
# generate start selling week
selling_weeks = df_weekly_sales.groupby(by='model_id',as_index=False).agg({'week_id':['min','max']})
selling_weeks.columns = ['model_id','start_selling_week','last_selling_week']
selling_weeks['length'] = selling_weeks.last_selling_week - selling_weeks.start_selling_week

In [12]:
df_tree.head(2)

,model_id,week_id,family_id,sub_department_id,department_id,univers_id,product_nature_id,model_label,family_label,sub_department_label,department_label,univers_label,product_nature_label,brand_label,brand_type
145,8356685,202138,34003,2088,403,8,25014,TEE SHIRT LIGHT M YELLOW,M INT BADM APP,BADMINTON INTER,BADMINTON SQUAS,RACKETS SPORTS,T-SHIRT,ARTENGO,MP
308,8315913,202138,34218,2143,117,35,25053,PA W EVOSTYLE SQUARRED PURPLE,WOMEN SNB CLTHG,SKI SNOWBOARD F,SKI FREESKI SNO,WINTER SPORTS U,TROUSERS,WEDZE,MP


In [23]:
tree_df = df_tree[['model_id','family_id','sub_department_id','department_id','univers_id','product_nature_id',
                   'brand_label','brand_type']]

In [19]:
# weekly price in model level
df_weekly_price = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                           'global/model_week_price', prefix_filename ='part-')
df_weekly_price = df_weekly_price.groupby(['model_id', 'week_id','date']).agg(average_price = ('average_price', 'mean')).reset_index()

In [20]:
price_df = df_weekly_price.groupby(by='model_id',as_index=False).agg({'average_price':'median'})

In [21]:
price_df.head(2)

,model_id,average_price
0,18259,59.0
1,94885,14.9


##### Size - not leveraging now

In [3]:
# Fetch SKU information
sku = read_multipart_parquet_s3('fcst-clean-prod/datalake', 'd_sku', prefix_filename ='part-')
sku['rank'] = sku.groupby(by='mdl_num_model_r3')['rs_technical_date'].rank(method='first',ascending=False)
sku['rank'] = sku['rank'].astype(int)

In [18]:
size_df = sku[(sku.mdl_num_model_r3.isna()==False) & (sku['rank'] < 2)][['mdl_num_model_r3','grid_size']].fillna('NO SIZE').drop_duplicates()
size_df.mdl_num_model_r3 = size_df.mdl_num_model_r3.apply(lambda x: int(x))
size_df.columns = ['model_id','size']

In [19]:
# Data Cleaning
## Remove useless information
#size_df['size'] = size_df['size'].astype(str)
#size_df['size'] = size_df['size'].replace('.',np.nan).replace(' .',np.nan).replace(' ',np.nan).fillna('NO SIZE')

In [20]:
size_df['size'].value_counts()

NO SIZE      558943
.             87938
 .            13208
M             11681
L             11294
              ...  
18 LITRES         1
16'5              1
K                 1
21/22             1
28 HOLES          1
Name: size, Length: 2068, dtype: int64

##### DSM Code

In [14]:
dsm_df = sku[(sku.mdl_num_model_r3.isna()==False) & (sku['rank'] < 2)][['mdl_num_model_r3','dsm_code']].drop_duplicates()
dsm_df.mdl_num_model_r3 = dsm_df.mdl_num_model_r3.apply(lambda x: int(x))
dsm_df.columns = ['model_id','dsm_code']

In [17]:
dsm_df.head(2)

,model_id,dsm_code
0,8663039,None
1,676051,None


##### Sports

In [21]:
# Get sports
sports_mapping = pd.read_csv('../additional_datasource/sports_mapping.csv')
sports_mapping = sports_mapping[['family_id','sports']]
sports_mapping.head(2)

,family_id,sports
0,11451,BASEBALL AMERICAN FO
1,147,BASEBALL AMERICAN FO


##### Color

In [22]:
d_item_universe_cube = pd.read_csv('result/d_item_universe_cube.csv')
color_df = d_item_universe_cube[['model_code','colors']].dropna().drop_duplicates()
color_df.columns = ['model_id','colors']

In [23]:
print(color_df.shape[0])
color_df.colors.value_counts()

7185


黑色                 1259
蓝色                  883
灰色                  602
粉红色                 383
白色                  339
                   ... 
紫红色|紫色                1
橘黄色|红色|绿色             1
棕色|灰色|米色|绿色|黄褐色       1
红色|绿色|蓝绿色|蓝色          1
枣红色|粉红色|红色            1
Name: colors, Length: 332, dtype: int64

In [24]:
# data cleaning
colors = color_df['colors'].str.split('|', expand=True)
colors = colors.stack().reset_index(level=1, drop=True)
colors.name = 'colors'
color_res = color_df.drop(['colors'], axis=1).join(colors)

In [25]:
color_map = {
    '粉红色':'红色',
    '枣红色':'红色',
    '紫红色':'红色',
    '蓝绿色':'蓝色',
    '赭色':'棕色',
    '黄褐色':'棕色',
    '橘黄色':'黄色',
    '米色':'白色'
}
color_res.colors = color_res.colors.replace(color_map)
color_res = color_res.drop_duplicates()

In [26]:
color_base = color_res[['model_id']].drop_duplicates()
for i in color_res.colors.unique():
    color_base = color_base.merge(color_res[color_res.colors == i],on='model_id',how='left')
    color_base = color_base.rename(columns = {'colors':f'is_{i}'})
    color_base[f'is_{i}'] = color_base[f'is_{i}'].where(color_base[f'is_{i}'].isna(),1).fillna(0)
    
color_base['color_cnt'] = color_base[[i for i in color_base.columns if i.startswith('is')]].sum(axis=1)

In [27]:
color_base.head(2)

,model_id,is_多色,is_黑色,is_灰色,is_红色,is_白色,is_蓝色,is_黄色,is_紫色,is_绿色,is_棕色,is_无色,color_cnt
0,8362957,1,0,0,0,0,0,0,0,0,0,0,1
1,8510763,0,1,0,0,0,0,0,0,0,0,0,1


##### Gender

In [28]:
genders_df = d_item_universe_cube[['model_code','genders']].dropna().drop_duplicates()
genders_df.columns = ['model_id','genders']

In [29]:
genders_df.genders.value_counts()

中性                   3005
男士                   1711
女士                   1439
女童|男童                1254
女士|男士                1094
女士|女婴|女童|男士|男婴|男童     383
女童                    177
男童                    175
男婴                     83
女婴|男婴                  72
小马驹|马                  58
女士|女童|男士|男童            43
女婴                     16
女士|女童                   8
男士|男童                   8
小马驹                     7
女婴|女童|男婴|男童             5
马                       3
女婴|女童                   3
女士|女童|男童                2
中性|女士|男士                2
成人                      2
女婴|女童|男童                1
Name: genders, dtype: int64

In [30]:
genders = genders_df['genders'].str.split('|', expand = True)
genders = genders.stack().reset_index(level=1,drop=True)
genders.name = 'genders'
genders_res = genders_df.drop(['genders'], axis=1).join(genders)

In [31]:
genders_res.genders.value_counts()

男士     3241
中性     3007
女士     2971
女童     1876
男童     1871
男婴      543
女婴      480
小马驹      65
马        61
成人        2
Name: genders, dtype: int64

In [32]:
genders_base = genders_res[['model_id']].drop_duplicates()
for i in genders_res.genders.unique():
    genders_base = genders_base.merge(genders_res[genders_res.genders == i],on='model_id',how='left')
    genders_base = genders_base.rename(columns = {'genders':f'is_{i}'})
    genders_base[f'is_{i}'] = genders_base[f'is_{i}'].where(genders_base[f'is_{i}'].isna(),1).fillna(0)
    
genders_base['genders_cnt'] = genders_base[[i for i in genders_base.columns if i.startswith('is')]].sum(axis=1)

In [33]:
genders_base.head(2)

,model_id,is_中性,is_女士,is_男士,is_女婴,is_男婴,is_女童,is_男童,is_小马驹,is_马,is_成人,genders_cnt
0,8383857,1,0,0,0,0,0,0,0,0,0,1
1,8383858,1,0,0,0,0,0,0,0,0,0,1


#### Part II - Join tables

In [24]:
features_df = {
    'price':[price_df,'model_id'],
    'tree':[tree_df,'model_id'],
    'dsm_code':[dsm_df,'model_id']
    #'color':[color_base,'model_id'],
    #'genders':[genders_base,'model_id'],
    #'sports':[sports_mapping,'family_id']
}

In [25]:
selling_len = 60

new_model_base = df_sales[['model_id']].drop_duplicates().merge(selling_weeks,on='model_id',how='left')
new_model_base = new_model_base[new_model_base.start_selling_week.apply(lambda x:get_week_id(x,selling_len)) > 202138]

print(f'There are {new_model_base.shape[0]} model with less than {selling_len} selling weeks')

There are 2365 model with less than 60 selling weeks


In [26]:
thres = 120

ref_model_base = df_weekly_sales[['model_id']].drop_duplicates().merge(selling_weeks,on='model_id',how='left')
ref_model_base = ref_model_base[(ref_model_base.length >= thres) & (ref_model_base.last_selling_week == 202137)]

print(f'There are {ref_model_base.shape[0]} model with at least {thres} selling weeks')

There are 8326 model with at least 120 selling weeks


In [37]:
print(f'the number of pairs would be {2365*8326}.')

the number of pairs would be 19690990.


In [27]:
def create_feature_pool(base,features_df):
    for df in features_df.keys():
        base = base.merge(features_df[df][0],on=features_df[df][1],how='left')
    return base

In [28]:
new_model_pool = create_feature_pool(new_model_base,features_df)
ref_model_pool = create_feature_pool(ref_model_base,features_df)

#### Write to S3 buckets

In [33]:
def write_df_to_parquet_on_s3(df, bucket, filename, index=False, verbose=False, region_name='eu-west-1'):
    """
    Write an in-memory pandas DataFrame to a parquet file on a S3 bucket
    :param dataframe: (pandas DataFrame) the data to save
    :param bucket: (string) S3 bucket name
    :param filename: (string) full path to the parquet file within the given bucket
    :param index: (bool) index value of the underlying 'to_parquet' function called
    """
    if verbose:
        print("Writing {} records to {}".format(len(df), to_uri(bucket, filename)))

    buffer = io.BytesIO()
    df.to_parquet(buffer, index=index)
    boto3.resource('s3', region_name=region_name).Object(bucket, filename).put(Body=buffer.getvalue())

In [34]:
# new_model_pool
write_df_to_parquet_on_s3(new_model_pool, 'fcst-workspace', 
                          'forecast-cn/fcst-refined-demand-forecast-dev/test_data/cold_start_dev/new_model_pool_202138.parquet')

# ref_model_pool
write_df_to_parquet_on_s3(ref_model_pool, 'fcst-workspace', 
                          'forecast-cn/fcst-refined-demand-forecast-dev/test_data/cold_start_dev/ref_model_pool_202138.parquet')

#### EDA

In [35]:
new_model_pool.isna().sum()

model_id              0
start_selling_week    0
last_selling_week     0
length                0
average_price         0
family_id             0
sub_department_id     0
department_id         0
univers_id            0
product_nature_id     0
brand_label           0
brand_type            0
dsm_code              9
dtype: int64

In [36]:
ref_model_pool.isna().sum()

model_id               0
start_selling_week     0
last_selling_week      0
length                 0
average_price          0
family_id              0
sub_department_id      0
department_id          0
univers_id             0
product_nature_id      0
brand_label            7
brand_type             0
dsm_code              26
dtype: int64

In [5]:
df_tree = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_tree', prefix_filename ='part-')
df_tree = df_tree[df_tree['week_id'] == 202138]

In [7]:
df_tree[df_tree.model_id == 8313985]

,model_id,week_id,family_id,sub_department_id,department_id,univers_id,product_nature_id,model_label,family_label,sub_department_label,department_label,univers_label,product_nature_label,brand_label,brand_type
800007,8313985,202138,2042,2861,471,7,25032,PARKA TAIGA 100 BLACK,WARM CLOTHS,CROSS HUNT CLOT,HUNTING AND SHO,HUNTING AND SHO,JACKET,SOLOGNAC,MP
